In [1]:
# 필요 라이브러리 import
import json
import urllib.request
import pandas as pd
import os
from bs4 import BeautifulSoup
from selenium import webdriver

# 네이버 검색 API key
client_id = '' # 본인이 할당받은 ID 입력
client_secret = '' # 본인이 할당받은 Secret 입력

for start_num in range(1, 1001, 30):
    # 네이버 검색 API 옵션
    start = start_num
    display = 30
    search_type = 'blog'
    # 지식인 'kin' 블로그 'blog'   뉴스 'news'

    search_text = '최저임금'

    # 리스트 선언
    url_list = []
    date_list = []
    content_list = []

    # 네이버 검색 API
    encText = urllib.parse.quote(search_text)

    url = 'https://openapi.naver.com/v1/search/' + search_type + '?display=' + str(display) + '&start=' + str(start) + '&sort=sim&query=' + encText

    request = urllib.request.Request(url)
    request.add_header('X-Naver-Client-Id', client_id)
    request.add_header('X-Naver-Client-Secret', client_secret)

    # 네이버 검색 API response result
    try:
        response = urllib.request.urlopen(request, timeout = 30)
    except Exception as e:
        print('bad request : ', e)

    rescode = response.getcode()
    if (rescode == 200):
        response_body = response.read()
        resp = response_body.decode('utf-8')
    else:
        print('Error Code:' + rescode)
        resp = ''

    # resp 있을 경우에만 실행
    if resp:
        # resp를 json을 사용하여 json Object로 변환
        j = json.loads(resp)

        # json Object 중에서 items 추출
        for item in j['items']:
            print('item : ', item)
            url = str(item['link']).replace('?Redirect=Log&amp;logNo=', '/')
            print(url)

            # item 안에 post_date가 없을 시(KeyError) None 처리
            try:
                post_date = item['postdate'] # 지식인, 블로그
            except KeyError as e:
                post_date = None
                # print('KeyError : ', e)

            try:
                # 웹드라이버 실행
                options = webdriver.ChromeOptions()
                options.add_argument('headless')

                driver = webdriver.Chrome(executable_path = 'webdriver/chromedriver.exe', chrome_options = options)
                driver.get(url)
                driver.switch_to.default_content()

                # 내용 선언
                contents = ''

                # 지식인

                # 블로그
                if not url.find('blog.naver.com') == -1:
                    driver.switch_to.frame('mainFrame')
                    page = driver.page_source
                    soup = BeautifulSoup(page, 'html.parser')

                    contents = soup.find('div', {'id': 'postViewArea'})

                    if not contents:
                        contents = soup.find('div', {'class': 'se-viewer'})

                    if not contents:
                        contents = soup.find('div', {'id': 'postListBody'})   

            except Exception as e:
                print(e)

            # 내용 html을 파싱
            content = ''
            # 네이버가 아닐시 태그가 다르기 때문에 오류 처리
            try:
                for string in contents.stripped_strings:
                    content += repr(string)
            except Exception as e:
                print('content none error', url)


            # 블로그
            content = content.replace('\\r\\n', ' ').replace(',', ' ').replace('\u200b', '').replace('\u30fb', '').replace('\xa0', '').replace('\xa9', '').replace('\u2573', '')      

            if not content == '':
                # 사이트 url
                url_list.append(url)
                date_list.append(post_date)
                content_list.append(content)

            # 드라이버 종료
            driver.quit()

        # 검색 단어명으로 내용을 엑셀 저장
        naver_info_df = {'content' : content_list, 'date' : date_list, 'url' : url_list}

        df = pd.DataFrame(naver_info_df)

        file_name = 'crawler_data/naver_' + search_type + '_' + search_text + '.csv'

        if not os.path.isfile(file_name):
            try:
                df.to_csv(file_name, index = False, encoding = 'ms949')
            except:
                pass
        else:
            try:
                df.to_csv(file_name, mode = 'a', index = False, header = False, encoding = 'ms949')
            except:
                pass

        # 결과 출력
        print('저장 완료!')


item :  {'title': '2020년 <b>최저임금</b>, 시급 미리 확인하기!', 'link': 'https://blog.naver.com/jjc369?Redirect=Log&logNo=221724641695', 'description': '주휴수당은 주휴일에 하루치 임금을 별도로 산정해서 지급하는 수당을 말합니다. 먼저... 2020년 <b>최저임금</b>으로 알아본 최저연봉으로는 21,543,720으로 작년 대비 601,920원 인상이라고 보시면 됩니다. 2020년... ', 'bloggername': '살림녀의 연애하는 신혼일기☞☜', 'bloggerlink': 'https://blog.naver.com/jjc369', 'postdate': '20191202'}
https://blog.naver.com/jjc369?Redirect=Log&logNo=221724641695


c:\programdata\anaconda3\lib\site-packages\ipykernel_launcher.py:73: DeprecationWarning: use options instead of chrome_options


item :  {'title': '2019년 <b>최저임금</b> 월급, 얼마나 되는가?', 'link': 'https://blog.naver.com/ssst9317?Redirect=Log&logNo=221678015280', 'description': '우리나라는 <b>최저임금</b>제도를 시행하고 있으며 이는 국가가 임금의 최저 수준을 정하고 사용자에게 이 수준 이상의 임금을 지급하도록 강제함으로써 저 임금 근로자는 보호한다는 것을 뜻합니다. 따라서... ', 'bloggername': '전지적 보험 시점', 'bloggerlink': 'https://blog.naver.com/ssst9317', 'postdate': '20191015'}
https://blog.naver.com/ssst9317?Redirect=Log&logNo=221678015280
item :  {'title': '<b>최저임금</b> 인상으로 오히려 저임금자 월급 줄어?│인터비즈', 'link': 'https://blog.naver.com/businessinsight?Redirect=Log&logNo=221419321563', 'description': "<b>최저임금</b> 인상이 저임금자 월급 낮춘다? 한은 임현준 연구위원, 송헌재 서울시립대 교수 등은 14일 발간한 'BOK 경제연구-<b>최저임금</b>이 고용구조에 미치는 영향' 보고서에서 &quot;산업 내 <b>최저임금</b> 미만자(영향자)가... ", 'bloggername': 'InterBiz', 'bloggerlink': 'https://blog.naver.com/businessinsight', 'postdate': '20181214'}
https://blog.naver.com/businessinsight?Redirect=Log&logNo=221419321563
item :  {'title': '2019년 최저시급, <b>최저임금</b> 8가지 포인트 정리', 'link': 'https://blog.naver.com/

item :  {'title': '2020년 <b>최저임금</b> 얼마나 인상됐나?', 'link': 'https://blog.naver.com/gcegoh5548?Redirect=Log&logNo=221610894800', 'description': '2020년 <b>최저임금</b>, 노동계에서는 사실상 임금삭감이나 다름 없는 참사라고 반발하는 분위기입니다. 노동부는 이에 대해 재심의 요청을 하지 않기로 했습니다. 대통령 역시 당초 공약이었던 시급 1만원을... ', 'bloggername': '기업의 바른 이치, 리치랩', 'bloggerlink': 'https://blog.naver.com/gcegoh5548', 'postdate': '20190809'}
https://blog.naver.com/gcegoh5548?Redirect=Log&logNo=221610894800
item :  {'title': '직장 내 괴롭힘 금지와 <b>최저임금</b> 8,590원', 'link': 'https://blog.naver.com/dailygaewon?Redirect=Log&logNo=221640583986', 'description': '최근 몇 년간 <b>최저임금</b>이 대폭 인상됨에 따라 중소·영세기업과 소상공인의 경영난이 심화된 점 등이 고려된 것으로 보인다. 이에 따라 2020년 최저 월 임금액은 1,795,310원(주 40시간, 월 209시간 주휴 포함... ', 'bloggername': '데일리개원', 'bloggerlink': 'https://blog.naver.com/dailygaewon', 'postdate': '20190917'}
https://blog.naver.com/dailygaewon?Redirect=Log&logNo=221640583986
item :  {'title': '2019년 <b>최저임금</b>, 주휴수당 계산법', 'link': 'https://blog.naver.com/wooriyo0n02?Redirect=Log&logN

item :  {'title': '2020년 일자리안정자금과 두루누리 사회보험, <b>최저임금</b>', 'link': 'https://blog.naver.com/savetax119?Redirect=Log&logNo=221730979160', 'description': '2020년 적용되는 <b>최저임금</b>과 일자리안정자금 및 두루누리에 대해 알아볼까 합니다. 1. <b>최저임금</b> - 시간당 8,350원에서 8,590원으로 인상되었습니다. 월 1,795,310원 (주40시간 근무기준이며, 월 약... ', 'bloggername': '세무회계 다온 - 세무사 박지은의 이지세무', 'bloggerlink': 'https://blog.naver.com/savetax119', 'postdate': '20191209'}
https://blog.naver.com/savetax119?Redirect=Log&logNo=221730979160
item :  {'title': '2020년 <b>최저임금</b> 알아보기', 'link': 'https://blog.naver.com/yeskrmc?Redirect=Log&logNo=221603613658', 'description': '<b>최저임금</b>제도는 근로자에 대하여 임금의 최저 수준을 보장하여 근로자의 생활안정과 노동력의 질적 향상을 꾀함으로써 국민 경제의 건전한 발전에 이바지하게 함을 목적으로 합니다. 국가가 노사 간의 임금... ', 'bloggername': '한국원자력환경공단(KORAD)', 'bloggerlink': 'https://blog.naver.com/yeskrmc', 'postdate': '20190802'}
https://blog.naver.com/yeskrmc?Redirect=Log&logNo=221603613658
item :  {'title': '중국은 지역별로 <b>최저임금</b>이 다르다?', 'link': 'https://blog.naver.com/chinasisa?Redirect

...

item :  {'title': '필립 코틀러의 다른 자본주의 (6) <b>최저임금</b>인상 왜 반대하세요?', 'link': 'https://blog.naver.com/thenanbiz?Redirect=Log&logNo=220346091635', 'description': '주장과 <b>최저임금</b> 인상이 갖는 긍적정 효과에 대해 말씀드릴게요. 우리나라의 최저시급은 5,580원이고... 그 결과 <b>최저임금</b> 인상이 끼친 부정적인 영향이 없다는 결론이 나왔습니다. 임금인상을 하지 않은 곳이나... ', 'bloggername': '더난출판 블로그', 'bloggerlink': 'https://blog.naver.com/thenanbiz', 'postdate': '20150430'}
https://blog.naver.com/thenanbiz?Redirect=Log&logNo=220346091635
item :  {'title': '헛다리짚은 정부의 <b>최저임금</b> 대책', 'link': 'https://blog.naver.com/cleanshim?Redirect=Log&logNo=221438271615', 'description': '<b>최저임금</b>이 2년에 29%나 폭등한데다 일 안해도 받는 주휴수당의 시간을 근로시간에 포함시켜야 하며 주 52시간을 지키라니 사실상 40% 이상의 임금인상이 되어버려 살 수 없다고 터져나오는 생존 비명이다.... ', 'bloggername': '섬기는 마음으로! 심재철', 'bloggerlink': 'https://blog.naver.com/cleanshim', 'postdate': '20190109'}
https://blog.naver.com/cleanshim?Redirect=Log&logNo=221438271615
item :  {'title': '&lt;최원길 칼럼 = 편의점主의 아우성-정말 <b>최저임금</b> 때문이야!!?&gt;', 'link': 'http://blog.daum.net/wkc

item :  {'title': '내년 <b>최저임금</b> 얼마일까?', 'link': 'https://blog.naver.com/parkmina0216?Redirect=Log&logNo=221721219761', 'description': '해보겠습니다 <b>최저임금</b>제란 국가가 노사간의 임금 결정과정에서 개입하여 임금의 최저수준을 정하고 사용자에게 이 수준 이상의 임금을 지급하도록 하는 법을 말합니다 저임금근로자에 대한 보호를... ', 'bloggername': '행복한 날', 'bloggerlink': 'https://blog.naver.com/parkmina0216', 'postdate': '20191128'}
https://blog.naver.com/parkmina0216?Redirect=Log&logNo=221721219761
item :  {'title': '알바생 21% <b>최저임금</b>도 못 받는다', 'link': 'https://blog.naver.com/chjc9?Redirect=Log&logNo=221450805219', 'description': '[weidʒ] 임금, 급료 ▪ hourly wage = hourly pay 시급 hovers below the minimum wage. <b>최저임금</b>... 보다 아래에 ▪ minimum [mínǝmǝm] 최저, 최소 한도 Another 32 percent replied they... ', 'bloggername': '영어뉴스 즐기기', 'bloggerlink': 'https://blog.naver.com/chjc9', 'postdate': '20190125'}
https://blog.naver.com/chjc9?Redirect=Log&logNo=221450805219
item :  {'title': '[영국 YMS/영국 워킹홀리데이/영국 워홀] 2018 <b>최저임금</b>... ', 'link': 'https://blog.naver.com/mayberin?Redirect

content none error http://blog.jinbo.net/imaginer/577
item :  {'title': "[조선일보 '19.10.08] 韓銀, 연구자도 모르게 <b>최저임금</b> 부작용... ", 'link': 'https://blog.naver.com/khchoo1224?Redirect=Log&logNo=221672700923', 'description': '&quot;한국은행, 보고서 발간하면서 연구자도 모르게 <b>최저임금</b> 부정적 효과 축소&quot;(조선일보 보도) 추경호 의원, “한국은행이 정부정책에 대한 연구자의 비판이 담긴 보고서를 내부 검열 과정에서 의도적으로 축소... ', 'bloggername': '경제전문가 추경호', 'bloggerlink': 'https://blog.naver.com/khchoo1224', 'postdate': '20191009'}
https://blog.naver.com/khchoo1224?Redirect=Log&logNo=221672700923
item :  {'title': '2015년 달라지는 <b>최저임금</b> 제도를 살펴보아요!', 'link': 'https://blog.naver.com/daelimuc?Redirect=Log&logNo=220227604022', 'description': '<b>최저임금</b>제도의 목적은 근로자에 대하여 임금의 최저수준을 보장하여 근로자의 생활안정과 노동력의 질적향상을 꾀함으로써 국민경제의 건전한 발전에 이바지하게 함을 목적으로 한다고 해요... ', 'bloggername': '대림대학교 공식 블로그', 'bloggerlink': 'https://blog.naver.com/daelimuc', 'postdate': '20150102'}
https://blog.naver.com/daelimuc?Redirect=Log&logNo=220227604022
item :  {'title': '<b>최저임금</b> 인상의 명암을 민생에서 느끼다.',

Message: no such frame
  (Session info: headless chrome=78.0.3904.108)
  (Driver info: chromedriver=2.38.552522 (437e6fbedfa8762dec75e2c5b3ddb86763dc9dcb),platform=Windows NT 10.0.17763 x86_64)

content none error https://blog.naver.com/bshwanim?Redirect=Log&logNo=221406219852
item :  {'title': '[트렌토리] #<b>최저임금</b>', 'link': 'https://blog.naver.com/maeyoungism?Redirect=Log&logNo=221333591044', 'description': '결국 <b>최저임금</b>이니 정치권의 부패니 을들의 전쟁이니 같은 남탓&amp;환경탓을 하기 보다는, 이런 상황에서도 어떻게 하면 외부변수가 대신할 수 없는 나만의 것으로 #최고임금 을 받을 수 있을지를 탐구하는 자세가... ', 'bloggername': '사당 비누,입욕제공방 VINSCENT | 빈센트', 'bloggerlink': 'https://blog.naver.com/maeyoungism', 'postdate': '20180806'}
https://blog.naver.com/maeyoungism?Redirect=Log&logNo=221333591044
Message: no such frame
  (Session info: headless chrome=78.0.3904.108)
  (Driver info: chromedriver=2.38.552522 (437e6fbedfa8762dec75e2c5b3ddb86763dc9dcb),platform=Windows NT 10.0.17763 x86_64)

content none error https://blog.naver.com/maeyoungism?Redirect=Log&logNo=

Message: no such frame
  (Session info: headless chrome=78.0.3904.108)
  (Driver info: chromedriver=2.38.552522 (437e6fbedfa8762dec75e2c5b3ddb86763dc9dcb),platform=Windows NT 10.0.17763 x86_64)

content none error https://blog.naver.com/hellopolicy?Redirect=Log&logNo=221336319497
item :  {'title': '<b>최저임금</b> 인상으로 사람 대신 앱을 쓴다!', 'link': 'https://blog.naver.com/simon9627?Redirect=Log&logNo=221384643421', 'description': "<b>최저임금</b> 인상, 사람 대신 앱으로 본죽, <b>최저임금</b> 때문일까? 배달 서비스 도입 중단... 포장 주문 앱으로 고객만족시킬 것 '본죽'의 배달 서비스 도입 검토를 중단하고, 대신 모바일과 온라인 사전 주문 시스템을... ", 'bloggername': '스마트메이커', 'bloggerlink': 'https://blog.naver.com/simon9627', 'postdate': '20181025'}
https://blog.naver.com/simon9627?Redirect=Log&logNo=221384643421
Message: no such frame
  (Session info: headless chrome=78.0.3904.108)
  (Driver info: chromedriver=2.38.552522 (437e6fbedfa8762dec75e2c5b3ddb86763dc9dcb),platform=Windows NT 10.0.17763 x86_64)

content none error https://blog.naver.com/simon9627?Redirect=Log&logNo=2213846